**필요한 모듈 설치**

In [ ]:
!pip install pdfminer.six
!pip install flair transformers torch sentencepiece kobert-transformers -q
!pip install -U huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 44.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 858.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**드라이브 마운트**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd

# 공유 폴더 내의 CSV 파일들이 위치한 디렉토리 경로
folder_path = '/content/drive/MyDrive/aid_jiwon/'

# 디렉토리 내의 모든 CSV 파일을 가져오기
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# 빈 데이터프레임 생성
df_all = pd.DataFrame()

# 각 CSV 파일을 읽어서 df_all에 추가
for file in csv_files:
    print(file)
    file_path = os.path.join(folder_path, file)
    print(f"읽는 중: {file_path}")  # 진행 상황 출력
    df = pd.read_csv(file_path, encoding='utf-8')  # CSV 파일 읽기

    # NaN 값 처리 및 텍스트 정제
    df = df.dropna(subset=['지침내용', '지침유형', '공사종류'])  # 필수 열에 NaN이 있는 행 제거
    df['지침내용'] = df['지침내용'].str.replace('\n', ' ').str.strip()  # 줄바꿈 제거 및 공백 정리

    # 데이터프레임 추가
    df_all = pd.concat([df_all, df], ignore_index=True)

# 각 분류 차원에 대한 별도 파일 생성
with open('type.txt', 'w', encoding='utf-8') as f:
    for _, row in df_all.iterrows():
        text = row['지침내용']
        type_category = row['지침유형']

        # 공백 제거: 중간에 공백이 포함된 경우 제거
        type_category = type_category.replace(" ", "")  # 공백 제거

        # __label__로 쓰기
        f.write(f"__label__{type_category} {text}\n")

with open('construction.txt', 'w', encoding='utf-8') as f:
    for _, row in df_all.iterrows():
        text = row['지침내용']
        construction_category = row['공사종류']
        f.write(f"__label__{construction_category} {text}\n")

# 처리된 데이터 수 출력
print(f"처리된 데이터 수: {len(df_all)}")


part4_sentence.csv
읽는 중: /content/drive/MyDrive/aid_jiwon/part4_sentence.csv
labels_강경민.csv
읽는 중: /content/drive/MyDrive/aid_jiwon/labels_강경민.csv
datalabels_최민혜2.csv
읽는 중: /content/drive/MyDrive/aid_jiwon/datalabels_최민혜2.csv
aid_5_데이터_박비원.csv
읽는 중: /content/drive/MyDrive/aid_jiwon/aid_5_데이터_박비원.csv
처리된 데이터 수: 844


# **모델 학습 부분**

In [ ]:
#### type.txt 분리 (라벨 별로 균등하게) ####
import random
from collections import defaultdict

# 파일 경로
input_file = '/content/type.txt'
train_file = 'type_train.txt'
dev_file = 'type_dev.txt'
test_file = 'type_test.txt'

# 비율 설정
train_ratio = 0.8
dev_ratio = 0.1
test_ratio = 0.1

# 라벨별로 문장을 분류하기 위한 딕셔너리
label_to_lines = defaultdict(list)

# 파일 읽기 및 라벨 분류
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip() == "":
            continue
        if not line.startswith("__label__"):
            continue
        label = line.split()[0]  # "__label__xxx" 형태에서 라벨만 추출
        label_to_lines[label].append(line)

# 분할 결과 저장 리스트
train_data, dev_data, test_data = [], [], []

# 각 라벨별로 비율에 맞게 분할
for label, lines in label_to_lines.items():
    random.shuffle(lines)  # 라벨 내 셔플
    n = len(lines)
    train_end = int(n * train_ratio)
    dev_end = train_end + int(n * dev_ratio)

    train_data.extend(lines[:train_end])
    dev_data.extend(lines[train_end:dev_end])
    test_data.extend(lines[dev_end:])

# 전체 셔플 (원하면 생략 가능)
random.shuffle(train_data)
random.shuffle(dev_data)
random.shuffle(test_data)

# 파일 저장
with open(train_file, 'w', encoding='utf-8') as f:
    f.writelines(train_data)

with open(dev_file, 'w', encoding='utf-8') as f:
    f.writelines(dev_data)

with open(test_file, 'w', encoding='utf-8') as f:
    f.writelines(test_data)

# 결과 출력
print(f"총 라벨 개수: {len(label_to_lines)}")
print(f"학습 데이터: {len(train_data)}개")
print(f"검증 데이터: {len(dev_data)}개")
print(f"테스트 데이터: {len(test_data)}개")
print(f"균등 분할된 데이터가 저장되었습니다.")


총 라벨 개수: 3
학습 데이터: 674개
검증 데이터: 83개
테스트 데이터: 87개
균등 분할된 데이터가 저장되었습니다.


In [ ]:
# 중복된 데이터가 있는지 확인
train_set = set(train_data)
dev_set = set(dev_data)
test_set = set(test_data)

# 학습, 검증, 테스트 데이터셋 간의 교집합을 확인하여 중복 여부를 확인
train_dev_intersection = train_set.intersection(dev_set)
train_test_intersection = train_set.intersection(test_set)
dev_test_intersection = dev_set.intersection(test_set)

# 중복 확인 출력
print(f"학습 데이터와 검증 데이터 간 교집합 크기: {len(train_dev_intersection)}")
print(f"학습 데이터와 테스트 데이터 간 교집합 크기: {len(train_test_intersection)}")
print(f"검증 데이터와 테스트 데이터 간 교집합 크기: {len(dev_test_intersection)}")


학습 데이터와 검증 데이터 간 교집합 크기: 0
학습 데이터와 테스트 데이터 간 교집합 크기: 0
검증 데이터와 테스트 데이터 간 교집합 크기: 0


In [ ]:
#### construction.txt 분리 (라벨 별로 균등하게) ####
import random
from collections import defaultdict

# 파일 경로
input_file = '/content/construction.txt'
train_file = 'construction_train.txt'
dev_file = 'construction_dev.txt'
test_file = 'construction_test.txt'

# 비율 설정
train_ratio = 0.8
dev_ratio = 0.1
test_ratio = 0.1

# 라벨별로 문장을 분류하기 위한 딕셔너리
label_to_lines = defaultdict(list)

# 파일 읽기 및 라벨 분류
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip() == "":
            continue
        if not line.startswith("__label__"):
            continue
        label = line.split()[0]  # "__label__xxx" 형태에서 라벨만 추출
        label_to_lines[label].append(line)

# 분할 결과 저장 리스트
train_data, dev_data, test_data = [], [], []

# 각 라벨별로 비율에 맞게 분할
for label, lines in label_to_lines.items():
    random.shuffle(lines)  # 라벨 내 셔플
    n = len(lines)
    train_end = int(n * train_ratio)
    dev_end = train_end + int(n * dev_ratio)

    train_data.extend(lines[:train_end])
    dev_data.extend(lines[train_end:dev_end])
    test_data.extend(lines[dev_end:])

# 전체 셔플 (원하면 생략 가능)
random.shuffle(train_data)
random.shuffle(dev_data)
random.shuffle(test_data)

# 파일 저장
with open(train_file, 'w', encoding='utf-8') as f:
    f.writelines(train_data)

with open(dev_file, 'w', encoding='utf-8') as f:
    f.writelines(dev_data)

with open(test_file, 'w', encoding='utf-8') as f:
    f.writelines(test_data)

# 결과 출력
print(f"총 라벨 개수: {len(label_to_lines)}")
print(f"학습 데이터: {len(train_data)}개")
print(f"검증 데이터: {len(dev_data)}개")
print(f"테스트 데이터: {len(test_data)}개")
print(f"균등 분할된 데이터가 저장되었습니다.")


총 라벨 개수: 7
학습 데이터: 673개
검증 데이터: 81개
테스트 데이터: 90개
균등 분할된 데이터가 저장되었습니다.


In [ ]:
# 중복된 데이터가 있는지 확인
train_set = set(train_data)
dev_set = set(dev_data)
test_set = set(test_data)

# 학습, 검증, 테스트 데이터셋 간의 교집합을 확인하여 중복 여부를 확인
train_dev_intersection = train_set.intersection(dev_set)
train_test_intersection = train_set.intersection(test_set)
dev_test_intersection = dev_set.intersection(test_set)

# 중복 확인 출력
print(f"학습 데이터와 검증 데이터 간 교집합 크기: {len(train_dev_intersection)}")
print(f"학습 데이터와 테스트 데이터 간 교집합 크기: {len(train_test_intersection)}")
print(f"검증 데이터와 테스트 데이터 간 교집합 크기: {len(dev_test_intersection)}")

학습 데이터와 검증 데이터 간 교집합 크기: 0
학습 데이터와 테스트 데이터 간 교집합 크기: 0
검증 데이터와 테스트 데이터 간 교집합 크기: 0


In [ ]:
# 필요한 라이브러리 및 모듈 임포트
import torch
from flair.data import Sentence
from flair.data import Corpus  # 텍스트 데이터 처리를 위한 Corpus 클래스
from flair.datasets import ClassificationCorpus  # 분류 작업을 위한 특화된 코퍼스 클래스
from flair.embeddings import TransformerDocumentEmbeddings  # 트랜스포머 기반 임베딩 (BERT 등)
from flair.models import TextClassifier  # 텍스트 분류 모델
from flair.trainers import ModelTrainer  # 모델 학습을 위한 트레이너
from sklearn.metrics import f1_score
import random
import numpy as np
from sklearn.model_selection import StratifiedKFold
import gc
import os
import tempfile

**지침 유형 분류 모델**

In [ ]:

def cross_validate_with_random_search(corpus, document_embeddings, n_trials=10, n_folds=5, label_type='class'):
    # 데이터를 분리하여 사용
    sentences = corpus.train
    texts = [s.to_plain_string() for s in sentences]
    labels = [s.get_label(label_type).value for s in sentences]

    # 계층적 폴드 분할 -> 균등한 레이블 분포를 갖는 폴드 생성
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    fold_indices = list(skf.split(texts, labels))

    # 하이퍼파라미터 범위 설정
    learning_rate_range = [0.0001, 0.01]  # 학습률 범위
    batch_sizes = [8, 16, 32]  # 배치 크기 옵션
    epochs_range = [5, 15]  # 에폭 범위

    best_score = 0
    best_params = None
    results = []

    # 랜덤 서치 시작
    for trial in range(n_trials):
        # 랜덤 하이퍼파라미터 생성
        lr = np.exp(random.uniform(np.log(learning_rate_range[0]), np.log(learning_rate_range[1])))
        batch_size = random.choice(batch_sizes)
        epochs = random.randint(epochs_range[0], epochs_range[1])

        params = {
            'learning_rate': lr,
            'batch_size': batch_size,
            'epochs': epochs
        }

        print(f"Trial {trial + 1}/{n_trials}: lr={lr:.4f}, batch_size={batch_size}, epochs={epochs}")

        fold_scores = []

        for fold_idx, (train_idx, val_idx) in enumerate(fold_indices):
            print(f"  Fold {fold_idx + 1}/{n_folds} training...")

            # 폴드에 해당하는 훈련/검증 문장 추출
            train_sentences = [sentences[i] for i in train_idx]
            val_sentences = [sentences[i] for i in val_idx]

            # 임시 파일로 훈련/검증 데이터를 저장
            with tempfile.TemporaryDirectory() as temp_dir:
                # 훈련 파일
                train_file = os.path.join(temp_dir, "train.txt")
                with open(train_file, 'w', encoding='utf-8') as f:
                    for s in train_sentences:
                        f.write(f"__label__{s.get_label(label_type).value} {s.to_plain_string()}\n")

                # 검증 파일
                dev_file = os.path.join(temp_dir, "dev.txt")
                with open(dev_file, 'w', encoding='utf-8') as f:
                    for s in val_sentences:
                        f.write(f"__label__{s.get_label(label_type).value} {s.to_plain_string()}\n")

                # 임시 코퍼스 생성
                temp_corpus = ClassificationCorpus(temp_dir, train_file="train.txt", dev_file="dev.txt", test_file=None)

                # 분류기 생성
                classifier = TextClassifier(document_embeddings,
                                            label_dictionary=temp_corpus.make_label_dictionary(label_type),
                                            label_type=label_type)

                # 트레이너 생성 및 학습
                trainer = ModelTrainer(classifier, temp_corpus)
                try:
                    result = trainer.train(
                        base_path=f"models/fold{fold_idx}",
                        learning_rate=lr,
                        mini_batch_size=batch_size,
                        max_epochs=epochs
                    )

                    # 학습된 모델 로드 후 평가
                    current_fold_model_path = os.path.join(f"models/fold{fold_idx}", "best-model.pt")
                    current_fold_model = TextClassifier.load(current_fold_model_path)

                    correct_predictions = 0
                    total_samples = 0

                    for sentence in val_sentences:
                        test_sentence = Sentence(sentence.to_plain_string())
                        true_label = sentence.get_label(label_type).value
                        current_fold_model.predict(test_sentence)
                        pred_label = test_sentence.get_label('class').value

                        if pred_label == true_label:
                            correct_predictions += 1
                        total_samples += 1

                    # 폴드의 정확도 계산
                    current_fold_accuracy = correct_predictions / total_samples if total_samples > 0 else 0
                    fold_scores.append(current_fold_accuracy)

                    print(f"  Fold {fold_idx + 1} accuracy: {current_fold_accuracy:.4f}")

                    # 메모리 정리
                    del current_fold_model
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    gc.collect()

                except Exception as e:
                    print(f"  Error: {e}")

        # 각 하이퍼파라미터 조합의 평균 정확도 계산
        avg_score = sum(fold_scores) / n_folds
        print(f"  Average performance: {avg_score:.4f}")

        results.append({
            'params': params,
            'score': avg_score,
            'fold_scores': fold_scores
        })

        # 최고 성능 업데이트
        if avg_score > best_score:
            best_score = avg_score
            best_params = params

    # 최종 결과 출력
    sorted_results = sorted(results, key=lambda x: x['score'], reverse=True)

    print("\n=== Random Search Results Summary ===")
    for i, result in enumerate(sorted_results[:5]):
        p = result['params']
        print(f"{i + 1}st: lr={p['learning_rate']:.4f}, batch_size={p['batch_size']}, epochs={p['epochs']} - score: {result['score']:.4f}")

    print(f"\nBest hyperparameters: {best_params}, score: {best_score:.4f}")
    return best_params, sorted_results


# 1. 교차검증을 통해 최적 하이퍼파라미터 찾기
type_corpus = ClassificationCorpus('/content/',
                                   train_file='type_train.txt',
                                   dev_file='type_dev.txt',
                                   test_file='type_test.txt')

document_embeddings = TransformerDocumentEmbeddings('klue/bert-base')

best_params, results = cross_validate_with_random_search(
    corpus=type_corpus,
    document_embeddings=document_embeddings,
    n_trials=5,  # 랜덤 서치 횟수
    n_folds=5     # 5-fold 교차 검증
)

# 2. 최적 하이퍼파라미터로 최종 학습 진행
best_classifier = TextClassifier(document_embeddings,
                                 label_dictionary=type_corpus.make_label_dictionary(label_type='class'),
                                 label_type='class')

trainer = ModelTrainer(best_classifier, type_corpus)

trainer.train(
    base_path="models/final_model",  # 최종 모델을 저장할 경로
    learning_rate=best_params['learning_rate'],  # 최적 학습률
    mini_batch_size=best_params['batch_size'],  # 최적 배치 사이즈
    max_epochs=best_params['epochs']  # 최적 에폭 수
)

# 3. 최종 모델 로드 후 테스트 데이터 평가
final_model = TextClassifier.load("models/final_model/best-model.pt")

# 테스트 데이터로 평가
# 1. 테스트 문장 가져오기 (라벨 포함된 원본 문장 사용)
test_sentences = type_corpus.test

# 2. 정확도 계산
correct_predictions = 0
total_samples = len(test_sentences)

for sentence in test_sentences:
    true_label = sentence.get_label('class').value

    final_model.predict(sentence)  # 모델이 예측한 라벨을 sentence에 붙임
    pred_label = sentence.get_label('class').value  # 예측 라벨 가져오기

    if pred_label == true_label:
        correct_predictions += 1

# 3. 정확도 출력
test_accuracy = correct_predictions / total_samples if total_samples > 0 else 0
print(f"Test Accuracy: {test_accuracy:.4f}")

2025-05-13 08:46:33,118 Reading data from /content
2025-05-13 08:46:33,119 Train: /content/type_train.txt
2025-05-13 08:46:33,120 Dev: /content/type_dev.txt
2025-05-13 08:46:33,122 Test: /content/type_test.txt
2025-05-13 08:46:33,176 Initialized corpus /content/ (label type name is 'class')
Trial 1/5: lr=0.0013, batch_size=16, epochs=15
  Fold 1/5 training...
2025-05-13 08:46:36,866 Reading data from /tmp/tmpj6blyxo5
2025-05-13 08:46:36,867 Train: /tmp/tmpj6blyxo5/train.txt
2025-05-13 08:46:36,869 Dev: /tmp/tmpj6blyxo5/dev.txt
2025-05-13 08:46:36,870 Test: None
2025-05-13 08:46:36,895 No test split found. Using 10% (i.e. 54 samples) of the train split as test data
2025-05-13 08:46:36,896 Initialized corpus /tmp/tmpj6blyxo5 (label type name is 'class')
2025-05-13 08:46:36,897 Computing label dictionary. Progress:


0it [00:00, ?it/s]
485it [00:00, 5096.00it/s]

2025-05-13 08:46:36,998 Dictionary created for label 'class' with 3 values: 예방조치 (seen 258 times), 점검사항 (seen 194 times), 위험경고 (seen 33 times)
2025-05-13 08:46:37,070 ----------------------------------------------------------------------------------------------------
2025-05-13 08:46:37,072 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(32001, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): 


/usr/local/lib/python3.11/dist-packages/flair/trainers/trainer.py:107: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/flair/trainers/trainer.py:545: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2025-05-13 08:46:37,628 epoch 1 - iter 3/31 - loss 1.14924125 - time (sec): 0.53 - samples/sec: 90.61 - lr: 0.001347 - momentum: 0.000000
2025-05-13 08:46:38,255 epoch 1 - iter 6/31 - loss 0.97126128 - time (sec): 1.16 - samples/sec: 82.95 - lr: 0.001347 - momentum: 0.000000
2025-05-13 08:46:38,817 epoch 1 - iter 9/31 - loss 0.94890550 - time (sec): 1.72 - samples/sec: 83.76 - lr: 0.001347 - momentum: 0.000000
2025-05-13 08:46:39,408 epoch 1 - iter 12/31 - loss 0.95128909 - time (sec): 2.31 - samples/sec: 83.11 - lr: 0.001347 - momentum: 0.000000
2025-05-13 08:46:40,139 epoch 1 - iter 15/31 - loss 0.97218179 - time (sec): 3.04 - samples/sec: 78.91 - lr: 0.001347 - momentum: 0.000000
2025-05-13 08:46:40,662 epoch 1 - iter 18/31 - loss 0.98273801 - time (sec): 3.56 - samples/sec: 80.80 - lr: 0.001347 - momentum: 0.000000
2025-05-13 08:46:41,232 epoch 1 - iter 21/31 - loss 1.03273173 - time (sec): 4.13 - samples/sec: 81.28 - lr: 0.001347 - momentum: 0.000000
2025-05-13 08:46:41,753 epoch 

...

2025-05-13 09:39:25,377 DEV : loss 0.9582543969154358 - f1-score (micro avg)  0.7349
2025-05-13 09:39:25,418  - 1 epochs without improvement


2025-05-13 09:39:28,908 ----------------------------------------------------------------------------------------------------
2025-05-13 09:39:28,919 Loading model from best epoch ...


100%|██████████| 2/2 [00:00<00:00,  3.55it/s]

2025-05-13 09:39:31,880 
Results:
- F-score (micro) 0.7356
- F-score (macro) 0.6313
- Accuracy 0.7356

By class:
              precision    recall  f1-score   support

        예방조치     0.7018    0.8889    0.7843        45
        점검사항     0.7857    0.6471    0.7097        34
        위험경고     1.0000    0.2500    0.4000         8

    accuracy                         0.7356        87
   macro avg     0.8292    0.5953    0.6313        87
weighted avg     0.7620    0.7356    0.7198        87

2025-05-13 09:39:31,881 ----------------------------------------------------------------------------------------------------


Test Accuracy: 0.7356


In [ ]:
#### 지침 유형 분류 최종 학습 후, 진짜 최종 모델 저장 ####
type_corpus = ClassificationCorpus('/content/',
                                   train_file='type_train.txt',
                                   dev_file='type_dev.txt',
                                   test_file='type_test.txt')

document_embeddings = TransformerDocumentEmbeddings('klue/bert-base')

# 2. 최적 하이퍼파라미터로 최종 학습 진행
best_classifier = TextClassifier(document_embeddings,
                                 label_dictionary=type_corpus.make_label_dictionary(label_type='class'),
                                 label_type='class')

trainer = ModelTrainer(best_classifier, type_corpus)

trainer.train(
    base_path="models/type-classifier",  # 최종 모델을 저장할 경로
    learning_rate=0.0007,  # 최적 학습률
    mini_batch_size=8,  # 최적 배치 사이즈
    max_epochs=15  # 최적 에폭 수
)

2025-05-13 07:57:28,814 Reading data from /content
2025-05-13 07:57:28,814 Train: /content/type_train.txt
2025-05-13 07:57:28,815 Dev: /content/type_dev.txt
2025-05-13 07:57:28,815 Test: /content/type_test.txt
2025-05-13 07:57:28,850 Initialized corpus /content/ (label type name is 'class')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

...

2025-05-13 08:00:15,716 DEV : loss 0.7571348547935486 - f1-score (micro avg)  0.6867
2025-05-13 08:00:15,750  - 2 epochs without improvement


2025-05-13 08:00:16,989 ----------------------------------------------------------------------------------------------------
2025-05-13 08:00:16,994 Loading model from best epoch ...


100%|██████████| 2/2 [00:00<00:00,  3.50it/s]

2025-05-13 08:00:20,520 
Results:
- F-score (micro) 0.7241
- F-score (macro) 0.5675
- Accuracy 0.7241

By class:
              precision    recall  f1-score   support

        예방조치     0.6780    0.8889    0.7692        45
        점검사항     0.8462    0.6471    0.7333        34
        위험경고     0.5000    0.1250    0.2000         8

    accuracy                         0.7241        87
   macro avg     0.6747    0.5536    0.5675        87
weighted avg     0.7273    0.7241    0.7029        87

2025-05-13 08:00:20,521 ----------------------------------------------------------------------------------------------------


{'test_score': 0.7241379310344828}

**공사 종류 분류 모델**

In [ ]:

def cross_validate_with_random_search(corpus, document_embeddings, n_trials=10, n_folds=5, label_type='class'):
    # 데이터를 분리하여 사용
    sentences = corpus.train
    texts = [s.to_plain_string() for s in sentences]
    labels = [s.get_label(label_type).value for s in sentences]

    # 계층적 폴드 분할 -> 균등한 레이블 분포를 갖는 폴드 생성
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    fold_indices = list(skf.split(texts, labels))

    # 하이퍼파라미터 범위 설정
    learning_rate_range = [0.0001, 0.01]  # 학습률 범위
    batch_sizes = [8, 16, 32]  # 배치 크기 옵션
    epochs_range = [5, 15]  # 에폭 범위

    best_score = 0
    best_params = None
    results = []

    # 랜덤 서치 시작
    for trial in range(n_trials):
        # 랜덤 하이퍼파라미터 생성
        lr = np.exp(random.uniform(np.log(learning_rate_range[0]), np.log(learning_rate_range[1])))
        batch_size = random.choice(batch_sizes)
        epochs = random.randint(epochs_range[0], epochs_range[1])

        params = {
            'learning_rate': lr,
            'batch_size': batch_size,
            'epochs': epochs
        }

        print(f"Trial {trial + 1}/{n_trials}: lr={lr:.4f}, batch_size={batch_size}, epochs={epochs}")

        fold_scores = []

        for fold_idx, (train_idx, val_idx) in enumerate(fold_indices):
            print(f"  Fold {fold_idx + 1}/{n_folds} training...")

            # 폴드에 해당하는 훈련/검증 문장 추출
            train_sentences = [sentences[i] for i in train_idx]
            val_sentences = [sentences[i] for i in val_idx]

            # 임시 파일로 훈련/검증 데이터를 저장
            with tempfile.TemporaryDirectory() as temp_dir:
                # 훈련 파일
                train_file = os.path.join(temp_dir, "train.txt")
                with open(train_file, 'w', encoding='utf-8') as f:
                    for s in train_sentences:
                        f.write(f"__label__{s.get_label(label_type).value} {s.to_plain_string()}\n")

                # 검증 파일
                dev_file = os.path.join(temp_dir, "dev.txt")
                with open(dev_file, 'w', encoding='utf-8') as f:
                    for s in val_sentences:
                        f.write(f"__label__{s.get_label(label_type).value} {s.to_plain_string()}\n")

                # 임시 코퍼스 생성
                temp_corpus = ClassificationCorpus(temp_dir, train_file="train.txt", dev_file="dev.txt", test_file=None)

                # 분류기 생성
                classifier = TextClassifier(document_embeddings,
                                            label_dictionary=temp_corpus.make_label_dictionary(label_type),
                                            label_type=label_type)

                # 트레이너 생성 및 학습
                trainer = ModelTrainer(classifier, temp_corpus)
                try:
                    result = trainer.train(
                        base_path=f"models/fold{fold_idx}",
                        learning_rate=lr,
                        mini_batch_size=batch_size,
                        max_epochs=epochs
                    )

                    # 학습된 모델 로드 후 평가
                    current_fold_model_path = os.path.join(f"models/fold{fold_idx}", "best-model.pt")
                    current_fold_model = TextClassifier.load(current_fold_model_path)

                    correct_predictions = 0
                    total_samples = 0

                    for sentence in val_sentences:
                        test_sentence = Sentence(sentence.to_plain_string())
                        true_label = sentence.get_label(label_type).value
                        current_fold_model.predict(test_sentence)
                        pred_label = test_sentence.get_label('class').value

                        if pred_label == true_label:
                            correct_predictions += 1
                        total_samples += 1

                    # 폴드의 정확도 계산
                    current_fold_accuracy = correct_predictions / total_samples if total_samples > 0 else 0
                    fold_scores.append(current_fold_accuracy)

                    print(f"  Fold {fold_idx + 1} accuracy: {current_fold_accuracy:.4f}")

                    # 메모리 정리
                    del current_fold_model
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    gc.collect()

                except Exception as e:
                    print(f"  Error: {e}")

        # 각 하이퍼파라미터 조합의 평균 정확도 계산
        avg_score = sum(fold_scores) / n_folds
        print(f"  Average performance: {avg_score:.4f}")

        results.append({
            'params': params,
            'score': avg_score,
            'fold_scores': fold_scores
        })

        # 최고 성능 업데이트
        if avg_score > best_score:
            best_score = avg_score
            best_params = params

    # 최종 결과 출력
    sorted_results = sorted(results, key=lambda x: x['score'], reverse=True)

    print("\n=== Random Search Results Summary ===")
    for i, result in enumerate(sorted_results[:5]):
        p = result['params']
        print(f"{i + 1}st: lr={p['learning_rate']:.4f}, batch_size={p['batch_size']}, epochs={p['epochs']} - score: {result['score']:.4f}")

    print(f"\nBest hyperparameters: {best_params}, score: {best_score:.4f}")
    return best_params, sorted_results


# 1. 교차검증을 통해 최적 하이퍼파라미터 찾기
type_corpus = ClassificationCorpus('/content/',
                                   train_file='construction_train.txt',
                                   dev_file='construction_dev.txt',
                                   test_file='construction_test.txt')

document_embeddings = TransformerDocumentEmbeddings('klue/bert-base')

best_params, results = cross_validate_with_random_search(
    corpus=type_corpus,
    document_embeddings=document_embeddings,
    n_trials=5,  # 랜덤 서치 횟수
    n_folds=5     # 5-fold 교차 검증
)

# 2. 최적 하이퍼파라미터로 최종 학습 진행
best_classifier = TextClassifier(document_embeddings,
                                 label_dictionary=type_corpus.make_label_dictionary(label_type='class'),
                                 label_type='class')

trainer = ModelTrainer(best_classifier, type_corpus)

trainer.train(
    base_path="models/final_model",  # 최종 모델을 저장할 경로
    learning_rate=best_params['learning_rate'],  # 최적 학습률
    mini_batch_size=best_params['batch_size'],  # 최적 배치 사이즈
    max_epochs=best_params['epochs']  # 최적 에폭 수
)

# 3. 최종 모델 로드 후 테스트 데이터 평가
final_model = TextClassifier.load("models/final_model/best-model.pt")

# 테스트 데이터로 평가
# 1. 테스트 문장 가져오기 (라벨 포함된 원본 문장 사용)
test_sentences = type_corpus.test

# 2. 정확도 계산
correct_predictions = 0
total_samples = len(test_sentences)

for sentence in test_sentences:
    true_label = sentence.get_label('class').value

    final_model.predict(sentence)  # 모델이 예측한 라벨을 sentence에 붙임
    pred_label = sentence.get_label('class').value  # 예측 라벨 가져오기

    if pred_label == true_label:
        correct_predictions += 1

# 3. 정확도 출력
test_accuracy = correct_predictions / total_samples if total_samples > 0 else 0
print(f"Test Accuracy: {test_accuracy:.4f}")


2025-05-13 08:05:12,495 Reading data from /content
2025-05-13 08:05:12,497 Train: /content/construction_train.txt
2025-05-13 08:05:12,498 Dev: /content/construction_dev.txt
2025-05-13 08:05:12,499 Test: /content/construction_test.txt
2025-05-13 08:05:12,537 Initialized corpus /content/ (label type name is 'class')
Trial 1/5: lr=0.0003, batch_size=32, epochs=12
  Fold 1/5 training...
2025-05-13 08:05:13,919 Reading data from /tmp/tmpnxf7b86f
2025-05-13 08:05:13,920 Train: /tmp/tmpnxf7b86f/train.txt
2025-05-13 08:05:13,922 Dev: /tmp/tmpnxf7b86f/dev.txt
2025-05-13 08:05:13,923 Test: None
2025-05-13 08:05:13,961 No test split found. Using 10% (i.e. 54 samples) of the train split as test data
2025-05-13 08:05:13,962 Initialized corpus /tmp/tmpnxf7b86f (label type name is 'class')
2025-05-13 08:05:13,963 Computing label dictionary. Progress:


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
0it [00:00, ?it/s]
484it [00:00, 5159.77it/s]

2025-05-13 08:05:14,064 Dictionary created for label 'class' with 7 values: 공통 (seen 132 times), 가설구조물 (seen 124 times), 교량 (seen 74 times), 굴착공사 (seen 65 times), 건축 (seen 51 times), 양중작업 (seen 35 times),  (seen 3 times)
2025-05-13 08:05:14,071 ----------------------------------------------------------------------------------------------------
2025-05-13 08:05:14,073 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(32001, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query)


/usr/local/lib/python3.11/dist-packages/flair/trainers/trainer.py:545: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2025-05-13 08:05:14,385 epoch 1 - iter 1/16 - loss 2.40146112 - time (sec): 0.29 - samples/sec: 109.80 - lr: 0.000315 - momentum: 0.000000
2025-05-13 08:05:14,860 epoch 1 - iter 2/16 - loss 2.36628711 - time (sec): 0.77 - samples/sec: 83.46 - lr: 0.000315 - momentum: 0.000000
2025-05-13 08:05:15,277 epoch 1 - iter 3/16 - loss 2.40398272 - time (sec): 1.18 - samples/sec: 81.10 - lr: 0.000315 - momentum: 0.000000
2025-05-13 08:05:15,770 epoch 1 - iter 4/16 - loss 2.36536050 - time (sec): 1.68 - samples/sec: 76.37 - lr: 0.000315 - momentum: 0.000000
2025-05-13 08:05:16,225 epoch 1 - iter 5/16 - loss 2.38082080 - time (sec): 2.13 - samples/sec: 75.08 - lr: 0.000315 - momentum: 0.000000
2025-05-13 08:05:16,587 epoch 1 - iter 6/16 - loss 2.40961512 - time (sec): 2.49 - samples/sec: 77.00 - lr: 0.000315 - momentum: 0.000000
2025-05-13 08:05:16,969 epoch 1 - iter 7/16 - loss 2.41331049 - time (sec): 2.88 - samples/sec: 77.90 - lr: 0.000315 - momentum: 0.000000
2025-05-13 08:05:17,416 epoch 1 -

...

2025-05-13 08:42:07,927 DEV : loss 2.884580612182617 - f1-score (micro avg)  0.6296
2025-05-13 08:42:07,966  - 3 epochs without improvement


2025-05-13 08:42:09,317 ----------------------------------------------------------------------------------------------------
2025-05-13 08:42:09,320 Loading model from best epoch ...


100%|██████████| 2/2 [00:00<00:00,  3.52it/s]

2025-05-13 08:42:12,059 
Results:
- F-score (micro) 0.7333
- F-score (macro) 0.7123
- Accuracy 0.7333

By class:
              precision    recall  f1-score   support

          공통     0.8889    0.6667    0.7619        24
       가설구조물     0.8333    0.6818    0.7500        22
          교량     0.6667    0.8571    0.7500        14
        굴착공사     0.7143    0.8333    0.7692        12
          건축     0.6364    0.7778    0.7000         9
        양중작업     0.5000    0.7143    0.5882         7
                 1.0000    0.5000    0.6667         2

    accuracy                         0.7333        90
   macro avg     0.7485    0.7187    0.7123        90
weighted avg     0.7644    0.7333    0.7363        90

2025-05-13 08:42:12,060 ----------------------------------------------------------------------------------------------------


Test Accuracy: 0.7333


In [ ]:
#### 공사 종류 분류 최종 학습 후, 진짜 최종 모델 저장 ####
type_corpus = ClassificationCorpus('/content/drive/MyDrive/aid_jiwon/',
                                   train_file='construction_train.txt',
                                   dev_file='construction_dev.txt',
                                   test_file='construction_test.txt')

document_embeddings = TransformerDocumentEmbeddings('klue/bert-base')

# 2. 최적 하이퍼파라미터로 최종 학습 진행
best_classifier = TextClassifier(document_embeddings,
                                 label_dictionary=type_corpus.make_label_dictionary(label_type='class'),
                                 label_type='class')

trainer = ModelTrainer(best_classifier, type_corpus)

trainer.train(
    base_path="models/final_model",  # 최종 모델을 저장할 경로
    learning_rate=0.0013,  # 최적 학습률
    mini_batch_size=32,  # 최적 배치 사이즈
    max_epochs=14  # 최적 에폭 수
)

2025-05-13 08:01:55,256 Reading data from /content/drive/MyDrive/aid_jiwon
2025-05-13 08:01:55,257 Train: /content/drive/MyDrive/aid_jiwon/construction_train.txt
2025-05-13 08:01:55,258 Dev: /content/drive/MyDrive/aid_jiwon/construction_dev.txt
2025-05-13 08:01:55,259 Test: /content/drive/MyDrive/aid_jiwon/construction_test.txt
2025-05-13 08:01:57,256 Initialized corpus /content/drive/MyDrive/aid_jiwon/ (label type name is 'class')
2025-05-13 08:01:58,465 Computing label dictionary. Progress:


0it [00:00, ?it/s]
675it [00:00, 5021.29it/s]

2025-05-13 08:01:58,606 Dictionary created for label 'class' with 7 values: 공통 (seen 183 times), 가설구조물 (seen 171 times), 교량 (seen 108 times), 굴착공사 (seen 94 times), 건축 (seen 66 times), 양중작업 (seen 48 times),  (seen 5 times)
2025-05-13 08:01:58,613 ----------------------------------------------------------------------------------------------------
2025-05-13 08:01:58,614 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(32001, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query


/usr/local/lib/python3.11/dist-packages/flair/trainers/trainer.py:545: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2025-05-13 08:01:59,157 epoch 1 - iter 2/22 - loss 2.47248948 - time (sec): 0.52 - samples/sec: 122.58 - lr: 0.001300 - momentum: 0.000000
2025-05-13 08:01:59,957 epoch 1 - iter 4/22 - loss 2.32450229 - time (sec): 1.32 - samples/sec: 96.80 - lr: 0.001300 - momentum: 0.000000
2025-05-13 08:02:00,774 epoch 1 - iter 6/22 - loss 2.29223533 - time (sec): 2.14 - samples/sec: 89.74 - lr: 0.001300 - momentum: 0.000000
2025-05-13 08:02:01,756 epoch 1 - iter 8/22 - loss 2.20556416 - time (sec): 3.12 - samples/sec: 82.02 - lr: 0.001300 - momentum: 0.000000
2025-05-13 08:02:02,709 epoch 1 - iter 10/22 - loss 2.14461352 - time (sec): 4.07 - samples/sec: 78.54 - lr: 0.001300 - momentum: 0.000000
2025-05-13 08:02:03,705 epoch 1 - iter 12/22 - loss 2.09019104 - time (sec): 5.07 - samples/sec: 75.73 - lr: 0.001300 - momentum: 0.000000
2025-05-13 08:02:04,779 epoch 1 - iter 14/22 - loss 2.04911137 - time (sec): 6.14 - samples/sec: 72.91 - lr: 0.001300 - momentum: 0.000000
2025-05-13 08:02:05,724 epoch 

...

2025-05-13 08:04:32,114 DEV : loss 1.3429428339004517 - f1-score (micro avg)  0.5238
2025-05-13 08:04:32,152  - 0 epochs without improvement
2025-05-13 08:04:32,153 saving best model


2025-05-13 08:04:36,952 ----------------------------------------------------------------------------------------------------
2025-05-13 08:04:36,956 Loading model from best epoch ...


100%|██████████| 2/2 [00:00<00:00,  5.03it/s]

2025-05-13 08:04:39,713 
Results:
- F-score (micro) 0.5647
- F-score (macro) 0.4223
- Accuracy 0.5647

By class:
              precision    recall  f1-score   support

          공통     0.6429    0.6923    0.6667        26
       가설구조물     0.5517    0.7619    0.6400        21
          교량     0.6000    0.5000    0.5455        18
        굴착공사     0.5000    0.5000    0.5000         8
          건축     0.2500    0.1429    0.1818         7
        양중작업     0.0000    0.0000    0.0000         5

    accuracy                         0.5647        85
   macro avg     0.4241    0.4328    0.4223        85
weighted avg     0.5277    0.5647    0.5396        85

2025-05-13 08:04:39,714 ----------------------------------------------------------------------------------------------------


{'test_score': 0.5647058823529412}

# **pdf 추출**

In [ ]:
# 필요한 라이브러리 설치
!pip install mecab-python3
!apt-get update
!apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl
!pip install konlpy
!apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!pip install pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.8/588.8 kB 9.6 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,665 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from collections import Counter
import pdfplumber
import concurrent.futures
from konlpy.tag import Mecab
import MeCab
import urllib.request

In [ ]:
pdf_folder = "/content/drive/MyDrive/aid_jiwon/pdfs/"  # 실제 본인 google drive 속 PDF들이 저장된 폴더 경로로 변경하세요
output_folder = "/content/drive/MyDrive/aid_jiwon/pdf_result"  # 실제 출력 폴더 경로로 변경하세요(본인 드라이브의 원하는 자리)

mecab = MeCab.Tagger()

# urllib 라이브러리에서 제공하는 한국어 불용어 목록 가져오기
url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ko/master/stopwords-ko.txt"
DEFAULT_STOP_WORDS = urllib.request.urlopen(url).read().decode('utf-8').splitlines()
custom_stop_words = list(DEFAULT_STOP_WORDS) + []  # 필요에 따라 수정하세요

# 병렬 처리 설정
batch_size = 5
max_workers = 4


In [ ]:
### 스터디장님 코드 그대로 들고 와서 실행해본 부분 ###
def find_section_for_text(text_position, sections):
    """텍스트 위치가 속한 섹션 찾기"""
    for i, section in enumerate(sections):
        section_start = section.get("start_position", 0)
        section_end = sections[i+1].get("start_position", float('inf')) if i+1 < len(sections) else float('inf')

        if section_start <= text_position < section_end:
            return {
                "number": section["number"],
                "title": section["title"]
            }

    return None

def identify_hazards(text):
    """위험요소 식별 함수"""
    hazard_keywords = {
    "낙하": ["낙하", "낙하물", "낙하사고", "낙하하는"],
    "추락": ["추락", "추락사고", "추락하는"],
    "붕괴": ["붕괴", "붕괴사고", "붕괴위험"],
    "감전": ["감전", "감전사고"],
    "협착": ["협착", "끼임", "협착사고", "압착"],
    "전도": ["전도", "넘어짐", "전도사고"],
    "미끄러짐": ["미끄러짐", "미끄러져", "미끄러질"],
    "충돌": ["충돌", "부딪힘", "접촉사고"],
    "베임": ["베임", "절단", "베이는", "베어짐"],
    "찔림": ["찔림", "찔리는", "찔려서"],
    "화재": ["화재", "불", "불꽃", "발화"],
    "비래": ["비래", "튀는", "날아오는"],
    "질식": ["질식", "산소결핍", "호흡곤란"],
    "중독": ["중독", "유해가스", "중독성", "흡입"],
    "화상": ["화상", "데임", "화끈거림"],
    "침하": ["침하", "꺼짐"],
    "전복": ["전복", "뒤집힘"],
    "도괴": ["도괴", "무너짐"],
    "파열": ["파열", "터짐", "깨짐"],
    "절단": ["절단", "잘림"],
    "압출사고": ["압출", "끼임", "눌림"],
    "폭발": ["폭발", "터짐", "폭파"],
    "위험": ["위험", "위험요인", "위험성", "위험상황"],
    "부등침하": ["부등침하", "기울어짐"],
    "강풍": ["강풍", "센바람", "돌풍"],
    "수평력": ["수평력", "측면힘"],
    "균열": ["균열", "금", "갈라짐"],
    "부식": ["부식", "녹", "산화"],
    "충격": ["충격", "세게 부딪힘"],
    "진동": ["진동", "떨림", "흔들림"],
    "소음": ["소음", "시끄러움", "소리"],
    "분진": ["분진", "먼지"],
    "유출": ["유출", "누출", "새어 나옴"],
    "유독가스": ["유독가스", "독성가스", "맹독가스"],
    "좌굴": ["좌굴", "눌림", "찌그러짐"],
    "고소작업": ["고소작업", "높은 곳", "고소", "높은 작업"],
    "밀폐공간": ["밀폐공간", "닫힌공간", "환기 안 되는 곳"],
    "협소": ["협소", "좁은 공간"],
    "절토사면": ["절토사면", "사면", "비탈면"],
    "궤도": ["궤도", "레일"]
}

    hazards = []

    for keyword in hazard_keywords:
        if keyword in text:
            # 위험요소 주변 컨텍스트 추출
            # context_pattern = re.compile(r'([^\.]*' + keyword + r'[^\.]*)')
            # matches = context_pattern.findall(text)
            # hazards.extend(matches)
            hazards.append(keyword)

    return hazards

def identify_measures(text):
    """안전조치 식별 함수"""
    # 조치사항은 주로 지침 문장 자체가 됨
    # 추가 분석으로 세부 조치만 추출 가능
    measure_patterns = [
    # 기본 조치 패턴 (~하여야 한다)
    r'[^.\n]*?(설치|점검|확인|조치|방지|예방|실시|금지|착용|유지|고정|배치|제거|표시)[^.\n]*?(하여야\s*한다|해야\s*한다|한다)',

    # 금지 및 예방 조항
    r'[^.\n]*?하지\s*않아야\s*한다',
    r'[^.\n]*?하지\s*말아야\s*한다',
    r'[^.\n]*?않도록\s*한다',
    r'[^.\n]*?되지\s*않도록\s*한다',

    # 지시 조항 (~되도록 한다 / 하도록 한다)
    r'[^.\n]*?되도록\s*한다',
    r'[^.\n]*?하도록\s*한다',

    # 상태 유지 관련
    r'[^.\n]*?유지\s*되어야\s*한다',
    r'[^.\n]*?확보\s*하여야\s*한다',

    # 실천 강조 (~점검한다 / 확인한다)
    r'[^.\n]*?점검\s*한다',
    r'[^.\n]*?확인\s*한다',
    r'[^.\n]*?준수\s*한다',
    r'[^.\n]*?실시\s*한다',
    ]


    measures = []
    for pattern in measure_patterns:
        matches = re.findall(pattern, text)
        measures.extend(matches)

    return measures if measures else [text]  # 기본적으로 전체 문장을 조치사항으로 반환

# def extract_keywords(text, stop_words): # preprocess 함수에서 호출
#     """MeCab을 사용하여 텍스트에서 키워드 추출"""
#     parsed = mecab.parse(text)  # pos() 대신 parse() 사용

#     # 파싱 결과 처리 - parse()는 문자열을 반환하므로 처리 필요
#     morphs = []
#     for line in parsed.split('\n'):
#         if line == 'EOS' or not line:
#             continue
#         parts = line.split('\t')
#         if len(parts) >= 2:
#             word = parts[0]
#             pos_info = parts[1].split(',')[0]  # 품사 정보
#             morphs.append((word, pos_info))

#     # 명사만 추출 (NNG: 일반명사, NNP: 고유명사)
#     # 2글자 이상인 명사만 추출
#     nouns = [word for word, pos in morphs if (pos.startswith('N') and len(word) >= 2)]

#     # 불용어 제거 및 빈도수 계산
#     filtered_words = [word for word in nouns if word not in stop_words]
#     word_counts = Counter(filtered_words)

#     return word_counts

def extract_keywords_from_safety_sections(guidelines, stop_words):
    """안전 관련 섹션의 가이드라인에서만 키워드 추출"""
    safety_text = ""

    # 안전 관련 키워드 확장
    safety_keywords = ["안전", "위험", "주의", "예방", "보호", "방지", "재해", "사고", "추락", "낙하" , "안전작업"]

    # 안전 관련 키워드가 포함된 섹션의 가이드라인만 선택
    for guideline in guidelines:
        if "section" in guideline and guideline["section"]:
            section_title = guideline["section"].get("title", "")
            text = guideline.get("text", "")

            if any(keyword in section_title for keyword in safety_keywords) or any(keyword in text for keyword in safety_keywords):
                if text:
                    safety_text += text + " "


    # 텍스트가 없는 경우 (안전 관련 섹션이 없을 경우)
    if not safety_text:
        # 모든 안전지침을 대상으로 함
        for guideline in guidelines:
            if "text" in guideline and guideline["text"]:
                safety_text += guideline["text"] + " "

    # 이하 코드는 동일하게 유지
    # MeCab을 사용하여 형태소 분석
    parsed = mecab.parse(safety_text)

    # 파싱 결과 처리
    morphs = []
    for line in parsed.split('\n'):
        if line == 'EOS' or not line:
            continue
        parts = line.split('\t')
        if len(parts) >= 2:
            word = parts[0]
            pos_info = parts[1].split(',')[0]
            morphs.append((word, pos_info))

    # 명사만 추출 (2글자 이상)
    nouns = [word for word, pos in morphs if (pos.startswith('N') and len(word) >= 2)]

    # 불용어 제거 및 빈도수 계산
    filtered_words = [word for word in nouns if word not in stop_words]
    word_counts = Counter(filtered_words)

    return word_counts, bool(safety_text and safety_text != "")



def extract_safety_guidelines(text, sections):
    """안전 관련 섹션에서만 안전지침 문장 추출"""
    guidelines = []

    # 안전 관련 키워드 정의
    safety_keywords = ["안전", "위험", "주의", "예방", "보호", "방지", "재해", "사고", "추락", "낙하" , "검토사항" , "준수사항", "안전작업"]

    # 안전 관련 섹션 필터링
    safety_sections = []
    for section in sections:
        # print('섹션들 전부 추출해보겠습니다.')
        # print(section)
        section_title = section.get("title", "")
        # print('이번에는 title들 전부 출력\n')
        #print(section_title)
        #print(type(section_title)) # -> class 'str'
        if any(keyword in section_title for keyword in safety_keywords):
            #print(keyword)
            print('이게 키워드로 설정된 상태로 검색했을때')
            print(section_title)
            print('\n')
            safety_sections.append(section)

    print('추출된 section 제목들 : ')

    for section in safety_sections:

      section_title = section.get("title", "") # 안전이라는 키워드가 안에 들어 있어도 추출되지 않는 df 에 출력되지 않는 이유 -> content가 없으니까 슈바 가이드라인이 없지

      print(section_title)

      print('\n')

    print('safety_sections 는 잘 생성 되었는가')
    # for i in range(len(safety_sections)):
    #   print(safety_sections[i])

    #print(safety_sections)

    # 안전 관련 섹션이 없을 경우 모든 섹션 사용
    if not safety_sections:
        print('비어있는  section 입력됨...')
        safety_sections = sections

    # 안전지침 패턴 (예: ~하여야 한다)
    guideline_patterns = [
      # 기존 패턴
      r'[^\.]*하여야\s*한다\.',
      r'[^\.]*해야\s*한다\.',
      r'[^\.]*금지하여야\s*한다\.',
      r'[^\.]*않도록\s*한다\.',

      # 추가적인 명령/지시 패턴
      r'[^\.]*하도록\s*한다\.',
      r'[^\.]*이어야\s*한다\.',
      r'[^\.]*되어야\s*한다\.',
      r'[^\.]*갖추어야\s*한다\.',
      r'[^\.]*필요하다\.',
      r'[^\.]*요구된다\.',

      # 전문용어와 결합된 패턴
      r'[^\.]*설치[^\.]*사용하여야\s*한다\.',
      r'[^\.]*시공[^\.]*하여야\s*한다\.',
      r'[^\.]*안전[^\.]*확보하여야\s*한다\.',

      # 복합 구문 패턴
      r'[^\.]*으며[^\.]*하여야\s*한다\.',
      r'[^\.]*하고[^\.]*하여야\s*한다\.',

      # 금지 패턴 추가
      r'[^\.]*해서는\s*안된다\.',
      r'[^\.]*해서는\s*안\s*된다\.',
      r'[^\.]*하지\s*말아야\s*한다\.',
      r'[^\.]*하지\s*않아야\s*한다\.',

      # 조건부 패턴
      r'[^\.]*경우[^\.]*하여야\s*한다\.',
      r'[^\.]*시에는[^\.]*하여야\s*한다\.',
      r'[^\.]*전에[^\.]*하여야\s*한다\.',
      r'[^\.]*후에[^\.]*하여야\s*한다\.',

      # 일반적인 지시 패턴
      r'[^\.]*원칙으로\s*한다\.',
      r'[^\.]*실시한다\.',
      r'[^\.]*점검한다\.',
      r'[^\.]*확인한다\.'
    ]

    combined_pattern = '|'.join(guideline_patterns)
    guideline_regex = re.compile(combined_pattern)

    # 안전 관련 섹션 내에서만 안전지침 문장 찾기
    for section in safety_sections: # section은 dictionary
        # 이 함수가 수행이 되면서 , 전체 섹션들 중 안전과 관련된 제목을 가진 섹션들 중에서 특정 안전 가이드라인 문장으로 끝나는 가이드라인을 추출 , 그 가이드라인 안에서 stopwords 빼고 명사들만 추출함.
        # 이렇게 해서 pdf 들 속에 있는 진정한 안전지침 관련 키워드 들을 추출함.
        print('현재 검사 대상 section 내용')
        print(section)
        section_content = section.get("content", "")

        if section_content is None:  # 괄호 제거, null 대신 None 사용
            print('엮시 없구만')

        section_info = {
            "number": section.get("number", ""),
            "title": section.get("title", "")
        }

        for match in guideline_regex.finditer(section_content):
            guideline_text = match.group(0).strip()

            # 위험요소 식별
            hazards = identify_hazards(guideline_text)

            # 조치사항 식별
            measures = identify_measures(guideline_text)

            guidelines.append({
                "text": guideline_text,
                "hazards": hazards,
                "measures": measures,
                "section": section_info,
                "is_safety_section": any(keyword in section_info["title"] for keyword in safety_keywords)
            })

            #print(guidelines)

    return guidelines

In [ ]:
def preprocess_pdf(pdf_path):
    """단일 PDF 파일 전처리 함수"""
    print(f"\n===== 처리 시작: {os.path.basename(pdf_path)} =====")
    document_info = {"path": pdf_path, "filename": os.path.basename(pdf_path)}
    sections = []

    try:
        with pdfplumber.open(pdf_path) as pdf:
            # 기본 문서 정보 추출
            if len(pdf.pages) > 0:
                print('pdf의 페이지 수는')
                print(len(pdf.pages))
                first_page_text = pdf.pages[0].extract_text() or ""
                print(f"첫 페이지 텍스트 추출 (처음 100자): {first_page_text[:100]}")
                # 문서 코드 추출 시도 (KOSHA GUIDE 패턴)
                code_match = re.search(r'KOSHA\s+GUIDE\s+([A-Z]\s*-\s*\d+\s*-\s*\d+)', first_page_text)
                if code_match:
                    document_info["code"] = code_match.group(1).strip()

                # 문서 제목 추출 시도 (코드 아래 줄)
                title_match = re.search(r'KOSHA\s+GUIDE.*?\n(.*?)\n', first_page_text, re.DOTALL)
                if title_match:
                    document_info["title"] = title_match.group(1).strip()

            # 모든 페이지 텍스트 추출
            all_text = ""
            for i,page in enumerate(pdf.pages):
                page_text = page.extract_text() or ""
                all_text += page_text + "\n\n"
                if i == 8:  # 첫 페이지만 확인
                    print(f"페이지 {i+1} 텍스트 길이: {len(page_text)} 자")

            print(f"전체 텍스트 길이: {len(all_text)} 자")
            document_info["total_text"] = all_text


            # 섹션 분리
            section_pattern = re.compile(r'(\d+\.(?:\d+)?)\s+([가-힣A-Za-z\s]+)')
            section_matches = list(section_pattern.finditer(all_text))
            print(f"검출된 섹션 수: {len(section_matches)}") # 출력값 확인해보니 섹션의 수는 정상적으로 출력됨.

            for match in section_pattern.finditer(all_text):
                section_number = match.group(1)
                section_title = match.group(2).strip()

                # 섹션 시작 위치
                start_pos = match.start()

                # 다음 섹션 찾기
                next_match = section_pattern.search(all_text, match.end())
                end_pos = next_match.start() if next_match else len(all_text)

                section_content = all_text[match.end():end_pos].strip()

                sections.append({
                    "number": section_number,
                    "title": section_title,
                    "content": section_content,
                    "level": 1 if "." not in section_number else 2,
                    "start_position": start_pos
                })

                section_matches = list(section_pattern.finditer(all_text))

            for i in range(len(section_matches)):
                print(section_matches[i]) # 전체 섹션 출력해보니 섹션 자체는 제대로 출력되는 중

# print(f"검출된 섹션 수: {len(section_matches)}")----------------------------------------------> 여기부터 해바라 일단 갑자기 분석 멈춰서 그것부터 해결하고 section이름에

# 안전이 들어가 있는 섹션들만 골라서 그 안의 가이드라인 , 키워드 추출이 완료됨 즉 pdf-유효한 section-가이드라인-키워드 : 이게 잘 형성되어 있음을 확인하고 다음으로 넘어가면 됨.

            # # 안전지침 문장 추출
            # safety_guidelines = extract_safety_guidelines(all_text, sections)
            # document_info["safety_guidelines"] = safety_guidelines

            # # 키워드 추출
            # keywords = extract_keywords(all_text, DEFAULT_STOP_WORDS)
            # document_info["keywords"] = keywords

             # 안전지침 문장 추출
            print('sections와 section_matches 가 다른건가')

            print(sections)
            # [{'number': '1.', 'title': '목 적\n이 지침은 산업안전보건기준에 관한 규칙', 'level': 2, 'start_position': 668} ,  ] 이렇게 리스트 안에 딕셔너리가 있는 형태

            safety_guidelines = extract_safety_guidelines(all_text, sections)
            document_info["safety_guidelines"] = safety_guidelines

            # 안전 관련 섹션의 가이드라인에서만 키워드 추출
            keywords, used_safety_sections = extract_keywords_from_safety_sections(safety_guidelines, DEFAULT_STOP_WORDS)
            document_info["keywords"] = keywords
            document_info["used_safety_sections_only"] = used_safety_sections  # 안전 섹션만 사용했는지 여부



        return document_info

    except Exception as e:
        print(f"Error processing {pdf_path}: {str(e)}")
        return {"path": pdf_path, "filename": os.path.basename(pdf_path), "error": str(e)}

In [ ]:
def analyze_pdf_folder_parallel(folder_path, stop_words=None, batch_size=5, max_workers=4, output_folder=None):
    """병렬 처리로 폴더 내 PDF 파일 분석"""
    if stop_words is None:
        stop_words = DEFAULT_STOP_WORDS

    # PDF 파일 목록 가져오기
    pdf_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith('.pdf')]

    if not pdf_files:
        print("PDF 파일을 찾을 수 없습니다.")
        return None, None

    results = []
    all_keywords = Counter()

    # 직렬 처리 (MeCab은 멀티프로세싱 환경에서 문제가 발생할 수 있음)
    for pdf_file in pdf_files:
        result = preprocess_pdf(pdf_file) # 여러개의 pdf들에 대해 각 pdf 마다 전처리( 각 pdf 파일 정보 , 메타데이터 , 전체 텍스트(이건 일단 주석처리) , 섹션 구조 정보 , 안전 지침 목록 , 키워드 빈도)

        if "error" not in result:
            results.append(result)

            # 키워드 카운터 업데이트
            if "keywords" in result:
                print(result["keywords"])
                all_keywords.update(result["keywords"])

    print('출력 좀 해바라')

    # 결과를 데이터프레임으로 변환
    if results:
        df = pd.DataFrame(results)

        # 결과 저장 (선택적)
        if output_folder:
            os.makedirs(output_folder, exist_ok=True)
            df.to_csv(os.path.join(output_folder, "pdf_analysis_results.csv"), index=False)

        return df, all_keywords

    return None, None

In [ ]:
def save_analysis_results(df, keyword_counter, output_folder):
    """분석 결과 저장"""
    os.makedirs(output_folder, exist_ok=True)

    # 데이터프레임 저장
    df.to_csv(os.path.join(output_folder, "pdf_analysis_results.csv"), index=False)

    # 키워드 빈도 저장
    keyword_df = pd.DataFrame({
        'keyword': [k for k, v in keyword_counter.most_common()],
        'frequency': [v for k, v in keyword_counter.most_common()]
    })
    keyword_df.to_csv(os.path.join(output_folder, "keyword_frequency.csv"), index=False)

    # 안전지침 추출 결과 저장
    guidelines = []
    for _, row in df.iterrows():
        if "safety_guidelines" in row and row["safety_guidelines"]:
            for guideline in row["safety_guidelines"]:
                guideline_info = {
                    "document": row.get("filename", ""),
                    "section_number": guideline.get("section", {}).get("number", ""),
                    "section_title": guideline.get("section", {}).get("title", ""),
                    "guideline_text": guideline.get("text", ""),
                    "hazards": "; ".join(guideline.get("hazards", [])),
                    "measures": "; ".join(map(str, guideline.get("measures", [])))
                }
                guidelines.append(guideline_info)

    if guidelines:
        guidelines_df = pd.DataFrame(guidelines)
        guidelines_df.to_csv(os.path.join(output_folder, "safety_guidelines.csv"), index=False)

    return guidelines_df

In [ ]:
# PDF 폴더 분석 실행
preprocessed_df, keyword_freq = analyze_pdf_folder_parallel(
    pdf_folder,
    stop_words=custom_stop_words,
    batch_size=batch_size,
    max_workers=max_workers,
    output_folder=output_folder
)

print(keyword_freq)

print('여기까지는 됨')


# 분석 결과 확인 및 저장
if preprocessed_df is not None and not preprocessed_df.empty:
    # 상위 30개 키워드 출력
    top_keywords = keyword_freq.most_common(30)
    print("\n상위 30개 키워드:")
    for keyword, count in top_keywords:
        print(f"{keyword}: {count}")

    print(f"\n총 처리된 PDF 파일 수: {len(preprocessed_df)}")
    print(f"총 추출된 고유 키워드 수: {len(keyword_freq)}")
else:
    print("처리된 PDF 파일이 없습니다.")


===== 처리 시작: 취약시기 건설현장 안전작업지침.pdf =====


pdf의 페이지 수는
31
첫 페이지 텍스트 추출 (처음 100자): KOSHA GUIDE
C - 113 - 2020
취약시기 건설현장 안전작업 지침
2020. 12.
한국산업안전보건공단
페이지 9 텍스트 길이: 946 자


전체 텍스트 길이: 25968 자
검출된 섹션 수: 20
<re.Match object; span=(51, 80), match='12.\n한국산업안전보건공단\n\n안전보건기술지침의 개요\n'>
<re.Match object; span=(1214, 1242), match='1. 목 적\n이 지침은 산업안전보건기준에 관한 규칙'>
<re.Match object; span=(1558, 1581), match='2. 적용범위\n이 지침은 취약시기인 해빙기'>
<re.Match object; span=(1697, 1707), match='3. 용어의 정의\n'>
<re.Match object; span=(3896, 3936), match='4. 일반적인 안전조치\n긴급 상황을 대비하여 공사 전 피난계획을 수립하고'>
<re.Match object; span=(4033, 4047), match='4.1 강풍에 따른 무너짐'>
<re.Match object; span=(4581, 4591), match='4.2 화재 예방\n'>
<re.Match object; span=(4593, 4602), match='2.1 일반사항\n'>
<re.Match object; span=(6389, 6395), match='2.2 용접'>
<re.Match object; span=(7010, 7022), match='2.3 폴리 우레탄 폼'>
<re.Match object; span=(8151, 8163), match='4.3 밀폐공간 작업\n'>
<re.Match object; span=(9609, 9629), match='5. 취약시기별 재해 예방 조치사항\n'>
<re.Match object; span=(9629, 9643), match='5.1 일반적인 안전조치\n'>
<re.Match object; span=(9645, 9652), match='1.1 해빙기'>
<re.Match object; span=(10281, 10289), match='1.2 동절기\n'>
<re.Match

pdf의 페이지 수는
15
첫 페이지 텍스트 추출 (처음 100자): KOSHA GUIDE
C - 43 - 2023
콘크리트공사의 안전보건작업 지침
2023. 8.
한 국 산 업 안 전 보 건 공 단
안전보건기술지침은 산업안전보건기준에 관한 규칙 등
페이지 9 텍스트 길이: 695 자


전체 텍스트 길이: 11297 자
검출된 섹션 수: 35
<re.Match object; span=(50, 135), match='8.\n한 국 산 업 안 전 보 건 공 단\n안전보건기술지침은 산업안전보건기준에 관한 규>
<re.Match object; span=(755, 761), match='1. 목적 '>
<re.Match object; span=(885, 893), match='2. 적용범위 '>
<re.Match object; span=(1011, 1021), match='3. 용어의 정의 '>
<re.Match object; span=(1132, 1151), match='4. 거푸집 및 거푸집동바리 공사 '>
<re.Match object; span=(1228, 1235), match='4.1 일반 '>
<re.Match object; span=(1359, 1366), match='4.2 재료 '>
<re.Match object; span=(1490, 1497), match='4.3 조립 '>
<re.Match object; span=(1620, 1627), match='4.4 점검 '>
<re.Match object; span=(1750, 1759), match='4.5 존치기간 '>
<re.Match object; span=(1874, 1881), match='4.6 해체 '>
<re.Match object; span=(2005, 2013), match='5. 철근공사 '>
<re.Match object; span=(2130, 2137), match='5.1 운반 '>
<re.Match object; span=(2258, 2265), match='5.2 가공 '>
<re.Match object; span=(2386, 2393), match='5.3 조립 '>
<re.Match object; span=(2513, 2526), match='6. 콘크리트 타설공사 '>
<re.Match object; span=(2624, 2631), match='6.

...

Counter({'작업': 17, '안전': 6, '차량': 5, '기계': 5, '중지': 4, '그림': 3, '위험': 3, '산업': 3, '보건': 3, '기준': 3, '건설': 3, '근로자': 3, '방지': 3, '장비': 3, '장치': 3, '수립': 2, '규칙': 2, '계획서': 2, '금지': 2, '조치': 2, '유도자': 2, '배치': 2, '검정': 2, '강풍': 2, '악천후': 2, '경우': 2, '충전': 2, '인근': 2, '순서': 1, '절차': 1, '조경': 1, '공사': 1, '현장': 1, '여건': 1, '설계': 1, '도서': 1, '도면': 1, '시방서': 1, '검토': 1, '계획': 1, '사전': 1, '평가': 1, '실시': 1, '잠재': 1, '요인': 1, '대책': 1, '별표': 1, '시작': 1, '관리': 1, '감독자': 1, '필요': 1, '사항': 1, '점검': 1, '하역': 1, '운반': 1, '중량물': 1, '취급': 1, '작성': 1, '지휘자': 1, '지정': 1, '지휘': 1, '사용': 1, '작업반': 1, '경내': 1, '관계': 1, '사람': 1, '출입': 1, '접촉': 1, '운행': 1, '승차': 1, '탑승': 1, '브레이크': 1, '경보등': 1, '후방': 1, '감지기': 1, '안전장치': 1, '설치': 1, '상태': 1, '과부하': 1, '방호': 1, '기관': 1, '합격': 1, '운전': 1, '자격': 1, '확인': 1, '식재': 1, '우려': 1, '기상': 1, '안정': 1, '날씨': 1, '철골': 1, '준용': 1, '전로': 1, '충전부': 1, '센티미터': 1})

===== 처리 시작: 철탑공사 안전보건기술지침.pdf =====


pdf의 페이지 수는
29
첫 페이지 텍스트 추출 (처음 100자): KOSHA GUIDE
C - 98 - 2014
철탑공사 안전보건기술지침
2014. 11.
한국산업안전보건공단
페이지 9 텍스트 길이: 203 자


전체 텍스트 길이: 16933 자
검출된 섹션 수: 9
<re.Match object; span=(46, 75), match='11.\n한국산업안전보건공단\n\n안전보건기술지침의 개요\n'>
<re.Match object; span=(626, 661), match='1. 목 적\n이 지침은 가공 송전선로를 위한 철탑공사의 자재운반'>
<re.Match object; span=(775, 789), match='2. 적용범위\n이 지침은 '>
<re.Match object; span=(882, 892), match='3. 용어의 정의\n'>
<re.Match object; span=(1805, 1820), match='4. 철탑 조립공법의 종류\n'>
<re.Match object; span=(3162, 3173), match='5. 철탑 시공순서\n'>
<re.Match object; span=(3590, 3606), match='6. 철탑 기초공사 작업안전\n'>
<re.Match object; span=(5935, 5952), match='7. 철탑 조립 공사 작업안전\n'>
<re.Match object; span=(11876, 11892), match='8. 전선 설치공사 작업안전\n'>
sections와 section_matches 가 다른건가
[{'number': '11.', 'title': '한국산업안전보건공단\n\n안전보건기술지침의 개요', 'content': '○ 작성자 : 한국안전학회 최명기\n○ 제정 경과\n- 2014년 8월 건설안전분야 제정위원회 심의(제정)\n○ 관련규격 및 자료\n- 유해․위험방지계획서(765kV울진-신태백 가공송전선로 ○○건설현장등)\n- 345kV 이하 가공 송전선로공사 표준 시공요령(한국전기공사협회)\n- 송전분야 시공기준(한국전력공사)\n- 765kV 송전선로 일반 시방서(한국전력공사 전력계통 건설처)\n- KEC 154kV 송전선공사(공사시방서, 특기시방서)\n- 송변전분야 위험성 Check List\n○ 관련

pdf의 페이지 수는
24
첫 페이지 텍스트 추출 (처음 100자): KOSHA GUIDE
C - 59 - 2022
지붕공사 안전보건작업 기술지침
2022. 3.
한 국 산 업 안 전 보 건 공 단
페이지 9 텍스트 길이: 611 자


전체 텍스트 길이: 14370 자
검출된 섹션 수: 35
<re.Match object; span=(49, 86), match='3.\n한 국 산 업 안 전 보 건 공 단\n\n안전보건기술지침의 개요\n'>
<re.Match object; span=(969, 1031), match='1. 목적\n이 지침은 지붕공사를 수행하는 작업과정에서 준수하여야 할 안전보건작업 기술지>
<re.Match object; span=(1033, 1053), match='2. 적용범위\n이 지침은 지붕의 골조'>
<re.Match object; span=(1099, 1109), match='3. 용어의 정의\n'>
<re.Match object; span=(1978, 1984), match='7.5 KN'>
<re.Match object; span=(2342, 2360), match='4. 지붕 위 작업안전 일반기준\n'>
<re.Match object; span=(2360, 2405), match='4.1 안전에 관한 일반사항\n모든 지붕작업에서는 안전한 작업공간이 마련되어야 한다'>
<re.Match object; span=(2624, 2676), match='4.2 안전작업에 필요한 시설 및 장비\n지붕 위에서 작업 시 안전 시설이나 장비는 다음>
<re.Match object; span=(2754, 2835), match='4.3 화재 시 대피\n지붕 위에서 하는 용접 등의 화기작업과 실내 활동에 기인한 화재 >
<re.Match object; span=(2879, 2938), match='4.4 자재 취급\n안전한 자재 취급은 지붕 위 작업에서 매우 중요하므로 이를 위한 고려>
<re.Match object; span=(3052, 3077), match='4.5 낙하물체\n지붕 위에서 낙하하는 지붕 판'>
<re.Match object; span=(3174, 3184), match='4.6 기상조건\n비'>
<re.Match object; span=(3388, 3418),

In [ ]:
# preprocessed_df의 첫 몇 행 확인하기
print("=" * 80)
print("preprocessed_df 샘플 (기본 문서 정보):")
print("=" * 80)
# 데이터프레임이 너무 넓을 수 있으므로 주요 열만 선택하여 표시
if not preprocessed_df.empty:
    sample_columns = ['filename', 'code', 'title']
    if 'safety_guidelines' in preprocessed_df.columns:
        # 안전지침 개수 열 추가
        preprocessed_df['guidelines_count'] = preprocessed_df['safety_guidelines'].apply(lambda x: len(x) if isinstance(x, list) else 0)
        sample_columns.append('guidelines_count')

    print(preprocessed_df[sample_columns].head())
else:
    print("preprocessed_df가 비어있습니다.")

# 안전지침 데이터프레임 생성 및 확인
print("\n" + "=" * 80)
print("guidelines_df 샘플 (추출된 안전지침):")
print("=" * 80)

# guidelines_df 생성
guidelines = []
for _, row in preprocessed_df.iterrows():
    if "safety_guidelines" in row and row["safety_guidelines"]:
        for guideline in row["safety_guidelines"]:
            guideline_info = {
                "document": row.get("filename", ""),
                "section_number": guideline.get("section", {}).get("number", ""),
                "section_title": guideline.get("section", {}).get("title", ""),
                "guideline_text": guideline.get("text", ""),
                "hazards": "; ".join(guideline.get("hazards", [])),
                "measures": "; ".join(map(str, guideline.get("measures", [])))
            }
            guidelines.append(guideline_info)

if guidelines:
    guidelines_df = pd.DataFrame(guidelines)
    # 텍스트 열의 내용이 너무 길 수 있으므로 출력을 위해 일부만 표시
    if 'guideline_text' in guidelines_df.columns:
        guidelines_df['guideline_text_preview'] = guidelines_df['guideline_text'].apply(lambda x: x[:100] + '...' if len(x) > 100 else x)

    # 주요 열만 선택하여 표시 -> 어떤 pdf에 대해 각 section 별로 guideline 들이 추출되고 그 guideline 들은 guidelines_df 에 저장이 된다.
    display_columns = ['document', 'section_title', 'guideline_text_preview']
    print(guidelines_df[display_columns])

    # 공종별 안전지침 수 확인
    print("\n" + "=" * 80)
    print("공종별 안전지침 수:")
    print("=" * 80)
    section_counts = guidelines_df['section_title'].value_counts().head(10)
    print(section_counts)
else:
    print("추출된 안전지침이 없습니다.")




preprocessed_df 샘플 (기본 문서 정보):
                                            filename            code  \
0       취약시기 건설현장 안전작업지침.pdf  C - 113 - 2020   
1       콘크리트공사의 안전보건작업 지침.pdf   C - 43 - 2023   
2  중소규모 건설업체 본사의 안전보건관리...   C - 86 - 2018   
3    지하매설물 굴착공사 안전작업 지침.pdf   C - 37 - 2011   
4  초고층 건축물공사(화재예방) 안전보건ᄌ...   C - 91 - 2015   

            title  guidelines_count  
0  C - 113 - 2020                 5  
1   C - 43 - 2023                 0  
2   C - 86 - 2018                 3  
3   C - 37 - 2011                27  
4   C - 91 - 2015                14  

guidelines_df 샘플 (추출된 안전지침):
                                         document  \
0    취약시기 건설현장 안전작업지침.pdf   
1    취약시기 건설현장 안전작업지침.pdf   
2    취약시기 건설현장 안전작업지침.pdf   
3    취약시기 건설현장 안전작업지침.pdf   
4    취약시ᄀ

In [ ]:
save_analysis_results(preprocessed_df, keyword_freq, output_folder)

,document,section_number,section_title,guideline_text,hazards,measures
0,취약시기 건설현장 안전작업지침.pdf,4.,일반적인 안전조치\n긴급 상황을 대비하여 공사 전 피난계획을 수립하고,", 적기에 피난 교육(경보 방법,\n피난경로, 집결지, 층별 소화기 위치와 사용법 ...",붕괴; 화재; 질식,"('실시', '하여야 한다')"
1,취약시기 건설현장 안전작업지침.pdf,2.3,동절기 안전조치,"(1) 폭설과 결빙에 대한 재해예방\n(가) 가설계단, 작업발판, 개구부 주위 및 ...",,"(1) 폭설과 결빙에 대한 재해예방\n(가) 가설계단, 작업발판, 개구부 주위 및 ..."
2,취약시기 건설현장 안전작업지침.pdf,2.3,동절기 안전조치,(나) 현장 내 가설도로에는 차량계 건설기계의 미끄럼 방지를 위하여 모래함이나 염\...,,(나) 현장 내 가설도로에는 차량계 건설기계의 미끄럼 방지를 위하여 모래함이나 염\...
3,취약시기 건설현장 안전작업지침.pdf,2.3,동절기 안전조치,"(라) 현장 내 처마 부위 또는 흙막이 지보공, 터널 수직구 상부 등에 고드름이 발...",낙하,"('금지', '하여야 한다')"
4,취약시기 건설현장 안전작업지침.pdf,2.3,동절기 안전조치,"(마) 철골작업(철탑, 비계 포함)의 경우 강설량이 시간당 1cm 이상인 경우 작업...",위험,"('제거', '하여야 한다')"
...,...,...,...,...,...,...
428,지붕공사 안전보건작업 기술지침.pdf,4.1,안전에 관한 일반사항\n모든 지붕작업에서는 안전한 작업공간이 마련되어야 한다,"지붕작업 시 다음과 같은\n안전작업을 위한 방법, 필요 장비, 작업자에 대한 세부사...",,"지붕작업 시 다음과 같은\n안전작업을 위한 방법, 필요 장비, 작업자에 대한 세부사..."
429,지붕공사 안전보건작업 기술지침.pdf,4.5,낙하물체\n지붕 위에서 낙하하는 지붕 판,", 도구, 고정부품, 지붕 타일, 고온의 아스팔트 덩어리\n등의 물질은 작업자 뿐만...",낙하; 위험,"('점검', '하여야 한다')"
430,지붕공사 안전보건작업 기술지침.pdf,5.3,낙하물 방지,"(1) 소규모 자재 운반기기를 지붕 단부에 설치하는 경우에는 작업발판 및 안전난간,...",낙하,"3 이상, 인양용 로프는 안전계수 5 이상을 확보하여야 한다"
431,지붕공사 안전보건작업 기술지침.pdf,5.3,낙하물 방지,- 8 -\n\nKOSHA GUIDE\nC - 59 - 2022\n(2) 지붕작업 ...,,"('금지', '하여야 한다')"
